In [ ]:
# Assignment 2.2

# Load this image dataset from hugging face -> frgfm/imagenette

# Create a streamlit/gradio based interface where user can search an
# image from this database by using either a text description or by
# uploading an image.

# The top 3 matching images should be displayed to the user.

# Hint: Use Clip embeddings to perform this search.

In [2]:
# imports
import random
from PIL import Image
import requests
from tqdm import tqdm
import numpy as np
import base64
from io import BytesIO

import torch
from datasets import load_dataset
from transformers import CLIPProcessor, CLIPModel
from langchain_huggingface import HuggingFaceEmbeddings
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist

import chromadb
from chromadb.config import Settings

/home/kush_210/Vettura-genai/vettura/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-02-04 21:48:48.413522: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
# Load this image dataset from hugging face -> frgfm/imagenette

dataset = load_dataset("frgfm/imagenette","full_size", split ="train")

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = processor.tokenizer

In [4]:
dataset

Dataset({
    features: ['image', 'label'],
    num_rows: 9469
})

In [5]:
# Function to get image embeddings
def get_image_embeddings(image):
    inputs = processor(images=image, return_tensors="pt")
     # Get embeddings for the image
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
    # Detach from the computational graph, then normalize the embeddings
    outputs = outputs.detach() 
    outputs = outputs.squeeze().tolist()
    return outputs

# Function to get text embeddings
def get_text_embeddings(text):
    inputs = processor(text=[text], return_tensors="pt")
    outputs = model.get_text_features(**inputs)
    return outputs

def image_to_base64(image):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format='JPEG')  # You can change the format (e.g., 'PNG')
    img_byte_arr = img_byte_arr.getvalue()
    return base64.b64encode(img_byte_arr).decode('utf-8')

In [6]:
image_to_base64(dataset[0]["image"])

'...'

In [7]:
img_embedding = get_image_embeddings(dataset[0]["image"])
# img_embedding.shape

In [8]:
txt_embedding = get_text_embeddings("a black cat")
txt_embedding.shape

torch.Size([1, 512])

In [117]:
# Create a new client for chromaDB
# client = chromadb.PersistentClient(path='./chromaDB',settings=Settings(allow_reset=True))
# client.reset()
# collection = client.create_collection(name="image_embeddings")

In [118]:
for i in tqdm(range(len(dataset)), desc="loading images"):
    image = dataset[i]['image']
    image_embeddings = get_image_embeddings(image)
    image_base64 = image_to_base64(image)  

    collection.add(
        ids=[str(i)],  # Unique ID for each sentence
        documents=[image_base64],  # The text data (document)
        embeddings=[image_embeddings],  # The corresponding embedding
        metadatas=[{"id": i, "label": dataset[i]["label"]}]  # Optional metadata
    )
    

: 

In [9]:
# get image embedding and store in vectoredb

# Create a new client for chromaDB
# Initialize Chroma client to load the data from the disk
client = chromadb.PersistentClient(path='./chromaDB1', settings=Settings(allow_reset=False))

# Load the collection from ChromaDB (ensure the collection name is the same as when it was created)
collection = client.get_collection("image_embeddings")


In [10]:
collection.count()

9469

In [12]:
# Create a streamlit/gradio based interface where user can search an
# image from this database by using either a text description or by
# uploading an image.

# The top 3 matching images should be displayed to the user.
%%writefile app.py
import streamlit as st
import chromadb
from chromadb.config import Settings
from PIL import Image
import base64
from io import BytesIO
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import cdist
import torch

if "query_text" not in st.session_state:
    st.session_state["query_text"] = None
if "uploaded_image" not in st.session_state:
    st.session_state["uploaded_image"] = None
if "result_image" not in st.session_state:
    st.session_state["result_image"] = None

# Initialize Chroma client to load the data from the disk
client = chromadb.PersistentClient(
    path="./chromaDB1", settings=Settings(allow_reset=False)
)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = processor.tokenizer


# Function to get image embeddings
def get_image_embeddings(image):
    inputs = processor(images=image, return_tensors="pt")
    # Get embeddings for the image
    with torch.no_grad():
        outputs = model.get_image_features(**inputs)
    # Detach from the computational graph, then normalize the embeddings
    outputs = outputs.detach()
    outputs = outputs.squeeze().tolist()
    return outputs


# Function to get text embeddings
def get_text_embeddings(text):
    inputs = processor(text=[text], return_tensors="pt")
    outputs = model.get_text_features(**inputs)
    outputs = outputs.detach()
    outputs = outputs.squeeze().tolist()
    return outputs


def image_to_base64(image):
    img_byte_arr = BytesIO()
    image.save(img_byte_arr, format="JPEG")  # You can change the format (e.g., 'PNG')
    img_byte_arr = img_byte_arr.getvalue()
    return base64.b64encode(img_byte_arr).decode("utf-8")


# Load the collection from ChromaDB (ensure the collection name is the same as when it was created)
collection = client.get_collection("image_embeddings")


st.title("Image Search Engine")
st.write("Search for images using text or upload an image")

query_text = st.text_input("Enter a text description or upload an image")
if query_text:
    st.session_state["query_text"] = query_text

# Create a file uploader for the image
uploaded_image = st.file_uploader("Upload an image", type=["jpg", "jpeg", "png"])
if uploaded_image is not None:
    st.session_state["uploaded_image"] = uploaded_image

if st.session_state["uploaded_image"] is not None:
    # Open the uploaded image
    uploaded_image = st.session_state["uploaded_image"]
    st.write("Searching for similar images...")
    image = Image.open(uploaded_image)
    image_embedding = get_image_embeddings(image)
    st.session_state["result_image"] = collection.query(
        query_embeddings=image_embedding, n_results=3
    )["documents"][0]
    st.session_state["uploaded_image"] = None


if st.session_state["query_text"] is not None:
    query_text = st.session_state["query_text"]
    st.write("Searching for similar images...")
    print("Query text", query_text)
    query_embedding = get_text_embeddings(query_text)

    st.session_state["result_image"] = collection.query(
        query_embeddings=query_embedding, n_results=3
    )["documents"][0]

    st.session_state["query_text"] = None

if st.session_state["result_image"] is not None:
    image_base64 = st.session_state["result_image"]
    for image in image_base64:
        image_data = base64.b64decode(image)
        image = Image.open(BytesIO(image_data))
        st.image(image, use_container_width=True)
    st.session_state["result_image"] = None


UsageError: Line magic function `%%writefile` not found.


In [11]:
# Check and kill any existing process on port 8501
!fuser -k 8501/tcp
!npm install -g localtunnel@2.0.2 -y
!streamlit run app.py & npx localtunnel --port 8501 &

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


OSError: Background processes not supported.